In [1]:
# Date: Dec 24
# Note: Using Gem (in place of OpenAI), followed plan...
# so can scape websites and produce a brochure based on that site. 
# Note: The notebook can be split in 1/2 (via a header)
# The 1st 1/2 is Ed's set up, etc.
# The 2nd half is just the code need to get the result.
# Pretty cool

# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
#from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

# ----- Replacing with Gem code -----
import os
import google.generativeai as genai

genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",

}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)

chat_session = model.start_chat(
  history=[  ])

message = "this is a test only"
response = chat_session.send_message(message)
print(response.text)

There might be a problem with your API key? Please visit the troubleshooting notebook!
Understood.  I await your instructions or input.



In [171]:
# # Experimenting to see if can get markdown output
# import os
# import google.generativeai as genai

# genai.configure(api_key=api_key)

# # Create the model
# generation_config = {
#   "temperature": 1,
#   "top_p": 0.95,
#   "top_k": 40,
#   "max_output_tokens": 8192,
#   "response_mime_type": "text/plain"
#   #"response_mime_type": "application/json",
# }

# model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# chat_session = model.start_chat(history=[])

# message = "how do i make a carrot cake - output as markdown"
# response = chat_session.send_message(message)

# # Print the response as Markdown
# markdown_output = f"```\n{response.text}\n```"  # Wrap in code block for Markdown formatting
# print(markdown_output)


In [172]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [174]:
link_system_prompt = """You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"""

link_system_prompt += """The quality of your answer will be judged by the following criteria;
                        - Your output will be valid dictionary, with the expected structure.
                        - Your output will NOT be 'Type of links: <class 'str'>'
                        - Your output will NOT include any leading content (i.e. "```json")
                        - Your output will NOT include any trailing content (i.e. "```")
                        - If the your output can not be parsed as valid json without error then that is a failure on your part \
                        - Your output will appear in the below format, any addtional content (as described above) is
                          strictly forbidden 
                            {
                                "links": [
                                    {"type": "about page", "url": "https://full.url/goes/here/about"},
                                    {"type": "careers page": "url": "https://another.full.url/careers"}
                                ]
                            }
                        """


In [175]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
The quality of your answer will be judged by the following criteria;
                        - Your output will be valid dictionary, with the expected structure.
                        - Your output will NOT be 'Type of links: <class 'str'>'
                        - Your output will NOT include any leading content (i.e. "```json")
                        - Your output will NOT include any trailing content (i.e. "```")
                        - If the your output can not be parsed as valid json without error then that is a failure on your part                         - Your output will appear in the below format, any addtional content (as described above) is
                          strictly forbidden 
                            {
              

In [176]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [177]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [178]:
# def get_links(url):
#     website = Website(url)
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)


# ----- trying to replace above with Gem code -----
def get_links(url):
    website = Website(url)
    # response = openai.chat.completions.create(
    #     model=MODEL,
    #     messages=[
    #         {"role": "system", "content": link_system_prompt},
    #         {"role": "user", "content": get_links_user_prompt(website)}
    #   ],
    #     response_format={"type": "json_object"}

    response = chat_session.send_message(link_system_prompt + get_links_user_prompt(website))
    #print(response.text)
    response_format={"type": "json_object"}


    #)
    #result = response.choices[0].message.content
    result = response.text
    #return json.loads(result)
    # return result
    # print(result)

    # # ------ Adding the clean up -----
    # # -- This is the functtion to clean up
    import ast
    import re

    def extract_python_code(code_str):
        # Remove the markdown code block markers (```)
        code_str = re.sub(r'```json|```', '', code_str)

        # Remove comments from the string
        code_str = re.sub(r'#.*', '', code_str)

        # Attempt to parse the remaining code to ensure it's valid Python
        try:
            # Use the ast.parse function to check if the code is valid Python syntax
            ast.parse(code_str)
            return code_str.strip()  # Return the code if it's valid
        except SyntaxError:
            return ""  # Return an empty string if the code is invalid
        
    #print(extract_python_code(result))  
    result = extract_python_code(result)
    return result
    print(result)

#get_links("https://huggingface.co")
get_links("https://edwarddonner.com")
qa = get_links("https://edwarddonner.com")

# -----
# response = chat_session.send_message(message)
# print(response.text)


In [179]:
# Check if stgring or dict, and convert to dict if poss
import json

# Example variable that could be a string or dictionary
variable = qa

# Check if the variable is a string
if isinstance(variable, str):
    try:
        # Attempt to convert the string to a dictionary
        variable = json.loads(variable)
        print("Successfully converted string to dictionary:", variable)
    except json.JSONDecodeError:
        print("The string is not in a valid JSON format.")
elif isinstance(variable, dict):
    print("The variable is already a dictionary:", variable)
else:
    print("The variable is neither a string nor a dictionary.")
    


Successfully converted string to dictionary: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}


In [180]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

#huggingface = Website("https://huggingface.co")
ed = Website("https://edwarddonner.com")
#huggingface.links

In [181]:
# get_links("https://huggingface.co")
# tst = get_links("https://huggingface.co")

get_links("https://edwarddonner.com")
tst = get_links("https://edwarddonner.com")

In [182]:
print(tst)

{
  "links": [
    {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"}
  ]
}


In [183]:
# # -- This is the functtion to clean up
# import ast
# import re

# def extract_python_code(code_str):
#     # Remove the markdown code block markers (```)
#     code_str = re.sub(r'```json|```', '', code_str)

#     # Remove comments from the string
#     code_str = re.sub(r'#.*', '', code_str)

#     # Attempt to parse the remaining code to ensure it's valid Python
#     try:
#         # Use the ast.parse function to check if the code is valid Python syntax
#         ast.parse(code_str)
#         return code_str.strip()  # Return the code if it's valid
#     except SyntaxError:
#         return ""  # Return an empty string if the code is invalid
    
# print(extract_python_code(tst))  
# d = extract_python_code(tst)
# print(d)



## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [184]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)


# ---------------- Make sure link is dict ----------------
    # Check if stgring or dict, and convert to dict if poss
    import json

    # Example variable that could be a string or dictionary
    variable = qa

    # Check if the variable is a string
    if isinstance(variable, str):
        try:
            # Attempt to convert the string to a dictionary
            variable = json.loads(variable)
            print("Successfully converted string to dictionary:", variable)
        except json.JSONDecodeError:
            print("The string is not in a valid JSON format.")
    elif isinstance(variable, dict):
        print("The variable is already a dictionary:", variable)
    else:
        print("The variable is neither a string nor a dictionary.")
    links = variable

# ---------------- Make sure link is dict ----------------


    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [186]:
#print(get_all_details("https://huggingface.co"))
print(get_all_details("https://edwarddonner.com"))


Successfully converted string to dictionary: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}
Landing page:
Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engag

In [187]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. \
Output is to be in markdown (i.e. a professional format, with bold headders, proper spacing between different sections, etc.)\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [188]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company."
    user_prompt += f"Please provide output as if it were a professional brochure with bold text for headings, content nicely layed out under headings, different content split out into sections, etc.)\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [189]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Successfully converted string to dictionary: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company.Please provide output as if it were a professional brochure with bold text for headings, content nicely layed out under headings, different content split out into sections, etc.)\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3-Base\nUpdated\n1 day ago\n•\n655\n•\n745\ndeepseek-ai/DeepSeek-V3\nUpdated\n1 day ago\n•\n431\nQwen/QVQ-72B-Preview\nUpdated\n3 days ago\n•\n5.61k\n•\n348\nanswerdotai/ModernBERT-base\nUpdated\n2 days ago\n•\n31.9k\n•\n500\nIamCreateAI/Ruyi-Mini-

In [190]:
# -- default --
# def create_brochure(company_name, url):
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
#           ],
#     )
#     result = response.choices[0].message.content
#     display(Markdown(result))



# -- replacing with gemini -- 
def create_brochure(company_name, url):
    response = chat_session.send_message(system_prompt + get_brochure_user_prompt(company_name, url))
    result = response.text
    display(Markdown(result))
   

In [191]:
create_brochure("HuggingFace", "https://huggingface.com")

Successfully converted string to dictionary: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}


# Hugging Face: The AI Community Building the Future

## **About Hugging Face**

Hugging Face is more than just a company; it's the vibrant heart of the machine learning community.  We're a platform where researchers, developers, and enthusiasts collaborate to create, share, and improve models, datasets, and applications, accelerating progress in AI.  Our mission is to democratize access to AI and empower individuals to build the future of machine learning.

At Hugging Face, we believe in open-source collaboration.  Our open-source stack, including tools like Transformers, Diffusers, and the Hugging Face Hub, provide a seamless environment for development and deployment.  We support a wide range of modalities, including text, image, video, audio, and 3D, making it easy to explore and build AI solutions across different domains.


## **What We Offer**

**For Customers:**

* **Models:** Access and utilize a vast library of over 400,000 pre-trained models, ready to integrate into your projects.
* **Datasets:** Explore a comprehensive collection of 100,000+ datasets for various tasks, fostering innovation and efficient development.
* **Spaces:** Deploy and showcase your applications with our easy-to-use Spaces platform.  Whether it's a simple demo or a complex application, we provide the infrastructure for seamless deployment.
* **Compute:** Leverage our optimized Inference Endpoints and GPU-powered Spaces for faster model deployment and training.  Pricing starts at just $0.60/hour for GPU access.
* **Enterprise Solutions:**  Our Enterprise offering is tailored for organizations needing enterprise-grade security, access controls, and dedicated support.


**For Investors:**

Hugging Face is a rapidly growing platform at the forefront of the AI revolution.  Our open-source approach fosters a large and engaged community, providing a robust foundation for future growth and innovation.  We provide powerful tools for both individuals and enterprise customers, positioning us for substantial market expansion.  Our commitment to open-source innovation and community engagement makes us a unique and compelling investment opportunity.



**For Recruits:**

Hugging Face offers a unique and exciting work environment.  We're a passionate team dedicated to building the future of AI. We are committed to creating a collaborative and inclusive culture where every team member feels valued and empowered to make a difference.  If you're passionate about machine learning and open-source development, we invite you to explore career opportunities at Hugging Face. Visit [https://apply.workable.com/huggingface/](https://apply.workable.com/huggingface/) to learn more and apply.



## **Our Impact**

More than 50,000 organizations, including prominent names like AI2, Meta, Amazon Web Services, Google, Intel, Microsoft, Grammarly, and Writer, are already leveraging the Hugging Face platform to advance their AI initiatives.


## **Our Open-Source Contributions**

Hugging Face is not just a platform; we are actively contributing to the development of fundamental machine learning tools.  Our open-source projects, including Transformers, Diffusers, Safetensors, and many others, are widely adopted by the community and contribute significantly to the advancement of the field.  These tools have garnered millions of downloads, demonstrating their impact on the AI landscape.


## **Contact Us**

Visit us at [https://huggingface.com](https://huggingface.com) to learn more!


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

# t1s: just the key parts below for ease of reference:

In [ ]:
# Code works, but could refine, esp looking at the 'qa' part 

In [12]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
#from openai import OpenAI

In [13]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

# ----- Replacing with Gem code -----
import os
import google.generativeai as genai

genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",

}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)

chat_session = model.start_chat(
  history=[  ])

message = "this is a test only"
response = chat_session.send_message(message)
print(response.text)

There might be a problem with your API key? Please visit the troubleshooting notebook!
Understood.  I'm ready for your test.  Please proceed.



In [14]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [15]:
link_system_prompt = """You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"""

link_system_prompt += """The quality of your answer will be judged by the following criteria;
                        - Your output will be valid dictionary, with the expected structure.
                        - Your output will NOT be 'Type of links: <class 'str'>'
                        - Your output will NOT include any leading content (i.e. "```json")
                        - Your output will NOT include any trailing content (i.e. "```")
                        - If the your output can not be parsed as valid json without error then that is a failure on your part \
                        - Your output will appear in the below format, any addtional content (as described above) is
                          strictly forbidden 
                            {
                                "links": [
                                    {"type": "about page", "url": "https://full.url/goes/here/about"},
                                    {"type": "careers page": "url": "https://another.full.url/careers"}
                                ]
                            }
                        """

In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
# def get_links(url):
#     website = Website(url)
#     response = openai.chat.completions.create(
#         model=MODEL,
#         messages=[
#             {"role": "system", "content": link_system_prompt},
#             {"role": "user", "content": get_links_user_prompt(website)}
#       ],
#         response_format={"type": "json_object"}
#     )
#     result = response.choices[0].message.content
#     return json.loads(result)


# ----- trying to replace above with Gem code -----
def get_links(url):
    website = Website(url)
    response = chat_session.send_message(link_system_prompt + get_links_user_prompt(website))
    response_format={"type": "json_object"}
    result = response.text

    # # ------ Adding the clean up -----
    # # -- This is the functtion to clean up
    import ast
    import re
    def extract_python_code(code_str):
        # Remove the markdown code block markers (```)
        code_str = re.sub(r'```json|```', '', code_str)

        # Remove comments from the string
        code_str = re.sub(r'#.*', '', code_str)

        # Attempt to parse the remaining code to ensure it's valid Python
        try:
            # Use the ast.parse function to check if the code is valid Python syntax
            ast.parse(code_str)
            return code_str.strip()  # Return the code if it's valid
        except SyntaxError:
            return ""  # Return an empty string if the code is invalid
        
    #print(extract_python_code(result))  
    result = extract_python_code(result)
    return result
    #print(result)

# #get_links("https://huggingface.co")
# get_links("https://edwarddonner.com")
qa = get_links("https://edwarddonner.com")

# -----
# response = chat_session.send_message(message)
# print(response.text)


In [18]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)


# ---------------- Make sure link is dict ----------------
    # Check if stgring or dict, and convert to dict if poss
    import json

    # Example variable that could be a string or dictionary
    variable = qa

    # Check if the variable is a string
    if isinstance(variable, str):
        try:
            # Attempt to convert the string to a dictionary
            variable = json.loads(variable)
            print("Successfully converted string to dictionary:", variable)
        except json.JSONDecodeError:
            print("The string is not in a valid JSON format.")
    elif isinstance(variable, dict):
        print("The variable is already a dictionary:", variable)
    else:
        print("The variable is neither a string nor a dictionary.")
    links = variable

# ---------------- Make sure link is dict ----------------


    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [19]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. \
Output is to be in markdown (i.e. a professional format, with bold headders, proper spacing between different sections, etc.)\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company."
    user_prompt += f"Please provide output as if it were a professional brochure with bold text for headings, content nicely layed out under headings, different content split out into sections, etc.)\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
# -- replacing with gemini -- 
def create_brochure(company_name, url):
    response = chat_session.send_message(system_prompt + get_brochure_user_prompt(company_name, url))
    result = response.text
    display(Markdown(result))
   

In [23]:
#create_brochure("HuggingFace", "https://huggingface.com")
#create_brochure("Nebula", "https://edwarddonner.com")
#create_brochure("News", "https://www.news.com.au/")
#create_brochure("Bank", "https://www.bendigobank.com.au/")
create_brochure("Jobs", "https://www.seek.com.au/")



Successfully converted string to dictionary: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}
Found links: {'links': [{'type': 'about page', 'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}


# SEEK: Connecting Australia to Opportunity

**A Leader in Recruitment and Career Development**

SEEK is Australia's number one jobs, employment, career, and recruitment site.  We connect job seekers with employers across a vast range of industries and locations, empowering individuals to find fulfilling careers and businesses to build thriving teams.  Our comprehensive platform offers resources for every stage of the career journey, from job searching and career advice to salary information and company exploration.

**Our Mission:** To help people find and get jobs they love.


**For Job Seekers:**

* **Extensive Job Listings:** Search millions of jobs across diverse industries and locations, using advanced filters to refine your search based on salary, location, job type, and more.
* **Personalized Recommendations:**  Receive tailored job recommendations based on your skills and experience, ensuring you see the most relevant opportunities.
* **Career Resources:** Access valuable career advice, salary information, and company profiles to make informed career decisions.
* **Easy Application Process:** Apply for jobs quickly and efficiently through our user-friendly platform.

**For Employers:**

* **Targeted Recruitment:**  Reach a wide pool of qualified candidates through our effective job posting tools.
* **Advanced Search Capabilities:** Find the perfect candidates for your open positions using advanced search filters and candidate matching technology.
* **Streamlined Hiring Process:** Manage the entire hiring process efficiently with SEEK's user-friendly employer tools.
* **Market Insights:** Access valuable data and analytics to gain a deeper understanding of the job market and make strategic hiring decisions.

**Our Culture:**

(Note: Information on SEEK's company culture is not provided in the given text.  To complete this section, further information from SEEK's website or other sources would be needed.)  This section could highlight employee values, work environment, diversity initiatives, and opportunities for professional growth.

**Careers at SEEK:**

(Note: While the provided text lists a "Careers" link, no specific details are included.  Further information is required to detail career opportunities and benefits at SEEK.)  This section would list current job openings, describe the work environment, and outline the benefits of working at SEEK (e.g., employee perks, training opportunities, growth potential).

**Connecting Communities:**

SEEK operates across numerous countries and regions, recognizing and supporting the diverse cultural landscape of the regions it serves.


**Contact Us:**

(Note:  A "Contact Us" link is provided, but further contact information is required to fully populate this section.)  This section would include contact details, email address, and potentially social media links.
